In [ ]:
from openai import OpenAI
import pandas as pd
import json
from pydantic import BaseModel, Field
from IPython.display import Markdown
import os
from dotenv import load_dotenv

load_dotenv()



True

In [ ]:
import phoenix as px
import os
from phoenix.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.semconv.trace import SpanAttributes
from opentelemetry.trace import Status, StatusCode
from openinference.instrumentation import TracerProvider

d:\Workspace\viettel\mini-project\eval\eval-agent-sql\eval\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import phoenix as px

session = px.launch_app()


C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\contextlib.py:142: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\contextlib.py:142: SAWarning: Skipped unsupported reflection of expression-based index ix_latency
  next(self.gen)


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


In [ ]:

from phoenix.otel import register

tracer_provider = register(
  project_name="sql-agent",
  endpoint="http://localhost:6006/v1/traces",
  protocol="grpc",
  
)
OpenAIInstrumentor().instrument(tracer_provider = tracer_provider)

OpenTelemetry Tracing Details
|  Phoenix Project: sql-agent
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [ ]:
tracer = tracer_provider.get_tracer(__name__)


In [ ]:
# prompt template for step 2 of tool 1
SQL_GENERATION_PROMPT = """
Generate an SQL query based on a prompt"," Do not reply with anything besides the SQL query","
The prompt is: {prompt}

The SQL query should be valid and executable on a PostgreSQL database","
The database schema is as follows:
{schema}
Only Select statements are allowed","

"""

In [7]:

client = OpenAI()

MODEL = "gpt-4o-mini"

In [ ]:
# code for step 2 of tool 1
@tracer.chain()
def generate_sql_query(prompt: str, schema: str) -> str:
    """Generate an SQL query based on a prompt"""
    formatted_prompt = SQL_GENERATION_PROMPT.format(prompt=prompt, 
                                                    schema=schema)

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    
    return response.choices[0].message.content

In [ ]:
# code for tool 1
@tracer.tool()
def lookup_sales_data(prompt: str) -> str:
    """Implementation of data lookup from postgres db using SQL"""
    try:
        import requests
        with tracer.start_as_current_span("prompt for retrieving schema context",openinference_span_kind="chain") as span:
            span.set_input(prompt)
            # Step 1: Get schema context from the context endpoint
            context_response = requests.post(
                "http://localhost:8000/context",
                json={"query": prompt}  # Fixed: prompt is now the direct value"," not inside another dict
            )
            
            if context_response.status_code != 200:
                return f"Error retrieving schema context: {context_response.text}"
            
            context_data = context_response.json()
            # Extract text from contexts to use as schema
            schema = context_data.get("contexts", [])
            span.set_output(value=schema)
        # Step 2: Generate SQL query based on prompt and schema
        sql_query = generate_sql_query(prompt, schema)
        print(f"Generated SQL Query: {sql_query}")  # Debugging output
        # Clean the response to make sure it only includes the SQL code
        sql_query = sql_query.strip()
        sql_query = sql_query.replace("```sql", "").replace("```", "")
        
        # Step 3: Execute the SQL query using the data endpoint
        data_response = requests.post(
            "http://localhost:8000/data",
            json={"query": sql_query}
        )
        
        if data_response.status_code != 200:
            return f"Error executing query: {data_response.text}"
        
        # Convert the response to a pandas DataFrame for formatting
        result_data = data_response.json()
        if not result_data["records"]:
            return "No results found for your query."
            
        result = pd.DataFrame(result_data["records"])
        
        return result.to_string()
    except Exception as e:
        return f"Error accessing data: {str(e)}"

In [10]:
# example_data = lookup_sales_data("Tiền phạt của tỉnh AGG năm 2024 theo từng tháng là bao nhiêu")
# print(example_data)

# Router

In [ ]:
# Define tools/functions that can be called by the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "lookup_sales_data",
            "description": "Look up data from postgres database using SQL",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {"type": "string", "description": "The unchanged prompt that the user provided",""}
                },
                "required": ["prompt"]
            }
        }
    }
]

# Dictionary mapping function names to their implementations
tool_implementations = {
    "lookup_sales_data": lookup_sales_data
}

In [ ]:
# code for executing the tools returned in the model's response
@tracer.chain()
def handle_tool_calls(tool_calls, messages):
    
    for tool_call in tool_calls:   
        function = tool_implementations[tool_call.function.name]
        function_args = json.loads(tool_call.function.arguments)
        result = function(**function_args)
        messages.append({"role": "tool", "content": result, "tool_call_id": tool_call.id})
        
    return messages

In [ ]:
SYSTEM_PROMPT = """
You are a helpful assistant that can answer questions about data and generate visualizations using tools","
"""

In [ ]:
def run_agent(messages):
    print("Running agent with messages:", messages)
    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
    if not any(
            isinstance(message, dict) and message.get("role") == "system" for message in messages
        ):
            system_prompt = {"role": "system", "content": SYSTEM_PROMPT}
            messages.append(system_prompt)

    while True:
        # Router Span
        print("Starting router call span")
        with tracer.start_as_current_span(
            "router_call", openinference_span_kind="chain",
        ) as span:
            span.set_input(value=messages)
            
            response = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                tools=tools,
            )
            messages.append(response.choices[0].message.model_dump())
            tool_calls = response.choices[0].message.tool_calls
            print("Received response with tool calls:", bool(tool_calls))
            span.set_status(StatusCode.OK)
    
            if tool_calls:
                print("Starting tool calls span")
                messages = handle_tool_calls(tool_calls, messages)
                span.set_output(value=tool_calls)
            else:
                print("No tool calls, returning final response")
                span.set_output(value=response.choices[0].message.content)
                return response.choices[0].message.content

In [ ]:
def start_main_span(messages):
    print("Starting main span with messages:", messages)
    
    with tracer.start_as_current_span(
        "AgentRun", openinference_span_kind="agent"
    ) as span:
        span.set_input(value=messages)
        ret = run_agent(messages)
        print("Main span completed with return value:", ret)
        span.set_output(value=ret)
        span.set_status(StatusCode.OK)
        return ret

In [17]:
result = start_main_span([{"role": "user", 
                           "content": "Tiền phạt của tỉnh AGG năm 2024 theo từng tháng là bao nhiêu"}])

Starting main span with messages: [{'role': 'user', 'content': 'Tiền phạt của tỉnh AGG năm 2024 theo từng tháng là bao nhiêu'}]
Running agent with messages: [{'role': 'user', 'content': 'Tiền phạt của tỉnh AGG năm 2024 theo từng tháng là bao nhiêu'}]
Starting router call span
Received response with tool calls: True
Starting tool calls span
Generated SQL Query: ```sql
SELECT
    thang,
    SUM(tong_phat) AS tong_tien_phat
FROM
    public."vcc_vhkt.vhkt_import_dt_tp_vtt_2"
WHERE
    nam = 2024
    AND ma_tinh = 'AGG'
GROUP BY
    thang
ORDER BY
    thang;
```
Starting router call span
Received response with tool calls: False
No tool calls, returning final response
Main span completed with return value: Dưới đây là tiền phạt của tỉnh AGG năm 2024 theo từng tháng:

| Tháng | Tổng Tiền Phạt (VNĐ)    |
|-------|-------------------------|
| 1     | 30,166,750,000          |
| 2     | 8,531,815,000           |
| 3     | 4,670,423,000           |
| 4     | 5,558,362,000           |
| 5     | 8,

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from tqdm import tqdm
agent_questions = [
    "Tổng tiền phạt năm 2024 là bao nhiêu",
    "Tổng tiền phạt của tỉnh AGG năm 2024 là bao nhiêu",
    "Tiền phạt của tỉnh AGG theo từng tháng năm 2024",
    "Tiền phạt KPI duy trì của AGG năm 2024 là bao nhiêu",
    "Tiền phạt KPI duy trì của AGG theo từng tháng năm 2024",
    "Tiền phạt KPI triển khai mới của AGG năm 2024 là bao nhiêu",
    "Tiền phạt KPI triển khai mới của AGG theo từng tháng năm 2024",
    "Tiền phạt lỗi ý thức, thái độ của AGG năm 2024 là bao nhiêu",
    "Tiền phạt lỗi ý thức thái độ của AGG theo từng tháng năm 2024",
    "Tiền phạt rời mạng của AGG năm 2024 là bao nhiêu",
    "Tiền phạt rời mạng của AGG theo từng tháng năm 2024",
    "Tỷ lệ PAKH toàn quốc trong năm 2024 là bao nhiêu",
    "Tỷ lệ PAKH toàn quốc trong năm 2024 theo từng tháng là bao nhiêu",
  "Tỷ lệ sự cố lặp lại toàn quốc trong năm 2024 là bao nhiêu",
    "Tỷ lệ sự cố lặp lại toàn quốc trong năm 2024 theo từng tháng là bao nhiêu",
    "Tỷ lệ sự cố lặp lại toàn quốc trong năm 2024 là bao nhiêu",
    "Tỷ lệ sự cố lặp lại toàn quốc trong năm 2024 theo từng tháng là bao nhiêu",
    "Tỷ lệ sự cố lặp lại của AGG năm 2024 là bao nhiêu",
    "Tỷ lệ sự cố lặp lại của AGG năm 2024 theo từng tháng là bao nhiêu?",
    "Số PAKH phát sinh năm 2024 theo từng tháng là bao nhiêu?",
    "Tỷ lệ xử lý sự cố trong 3h năm 2024 của Toàn quốc theo từng tháng là bao nhiêu?",
    "Số lượng sự cố phát sinh của AGG năm 2024 theo từng tháng là bao nhiêu?",
    # "Tỷ lệ xử lý sự cố trong 10h năm 2024 của Toàn quốc là bao nhiêu?",

]

for question in tqdm(agent_questions, desc="Processing questions"):

    try:
        ret= lookup_sales_data(question)
        # print(f"Result for question '{question}':")
    except Exception as e:
        print(f"Error processing question: {question}")
        print(e)
        continue

Processing questions:   0%|          | 0/22 [00:00<?, ?it/s]

Generated SQL Query: ```sql
SELECT 
    thang, 
    AVG(ty_le_pakh_10000tb_ngay_dich_vu_ftth) AS ty_le_pakh_ftth, 
    AVG(ty_le_pakh_10000tb_ngay_dich_vu_khac) AS ty_le_pakh_khac
FROM 
    public."vcc_vhkt.vhkt_import_kpi_duy_tri_cdbr_tinh"
WHERE 
    nam = 2024
GROUP BY 
    thang
ORDER BY 
    thang;
```
Error processing question: Tỷ lệ PAKH toàn quốc trong năm 2024 theo từng tháng là bao nhiêu
object str can't be used in 'await' expression
Generated SQL Query: ```sql
SELECT 
    SUM(tong_su_co_phat_sinh_co_hen) AS tong_su_co_phat_sinh_co_hen,
    SUM(tong_su_co_duoc_xu_ly_dung_hen) AS tong_su_co_duoc_xu_ly_dung_hen,
    (SUM(tong_su_co_duoc_xu_ly_dung_hen) * 100.0 / NULLIF(SUM(tong_su_co_phat_sinh_co_hen), 0)) AS ty_le_su_co_duoc_xu_ly_dung_hen
FROM 
    public."vcc_vhkt.vhkt_import_kpi_duy_tri_cdbr_tinh"
WHERE 
    nam = 2024;
```
Error processing question: Tỷ lệ sự cố lặp lại toàn quốc trong năm 2024 là bao nhiêu
object str can't be used in 'await' expression
Generated SQL Query:

In [ ]:
from phoenix.evals import (
    TOOL_CALLING_PROMPT_TEMPLATE, 
    llm_classify,
    OpenAIModel
)
from phoenix.trace import SpanEvaluations
from phoenix.trace.dsl import SpanQuery
from openinference.instrumentation import suppress_tracing

In [ ]:
query = SpanQuery().where(
    "name =='generate_sql_query'"
).select(
    sql_gen="output.value",
    context="input.value",
    
)
prompt = SpanQuery().where(
    "span_kind=='AGENT'"
).select(
    prompt="input.value"
)

# The Phoenix Client can take this query and return the dataframe.
sql_df = px.Client().query_spans(query, 
                                 project_name="sql-agent",
                                 timeout=None)

prompt_df = px.Client().query_spans(prompt,
                                    project_name="sql-agent",
                                    timeout=None)
sql_df['prompt'] = prompt_df['prompt'].values
sql_df.head()

,sql_gen,context,prompt
context.span_id,,,
715fd7cdad76baed,```sql\nSELECT SUM(tong_phat) AS tong_tien_pha...,"{""prompt"": ""Tổng tiền phạt năm 2024"", ""schema""...","[{""role"": ""user"", ""content"": ""Tổng tiền phạt n..."
ca196ca3dd66c0db,```sql\nSELECT SUM(tong_phat) AS tong_tien_pha...,"{""prompt"": ""Tổng tiền phạt của tỉnh AGG năm 20...","[{""role"": ""user"", ""content"": ""Tổng tiền phạt c..."
08394cc649b3b987,"```sql\nSELECT thang, SUM(tong_phat) AS tong_t...","{""prompt"": ""Tiền phạt của tỉnh AGG theo từng t...","[{""role"": ""user"", ""content"": ""Tiền phạt của tỉ..."
4aeb95b07ed2be60,```sql\nSELECT \n SUM(phat_kpi_duy_tri_cdbr...,"{""prompt"": ""Tiền phạt KPI duy trì của AGG năm ...","[{""role"": ""user"", ""content"": ""Tiền phạt KPI du..."
e8cad356d2362fe5,"```sql\nSELECT thang, SUM(phat_kpi_duy_tri_cdb...","{""prompt"": ""Tiền phạt KPI duy trì của AGG theo...","[{""role"": ""user"", ""content"": ""Tiền phạt KPI du..."


In [ ]:
# save sql_df to a file
sql_df.to_csv("sql_queries.csv", index=False)

In [ ]:
#save to excel 
sql_df.to_excel("sql_queries2.xlsx", index=False)